In [2]:
import pandas as pd
from osgeo import gdal
import math
import os
import glob
from PIL import Image # PIL library supports only TIF format but not newer TIFF
import numpy as np
import shutil

In [8]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
interpolated_AIS_data_directory = data_directory + '5_interpolated_AIS_data/'
#annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
final_dataset = data_directory + '7_final_dataset/'
final_dataset_images = final_dataset + 'images/'
final_dataset_labels = final_dataset + 'labels/'
###
second_final_dataset = data_directory + '8_final_dataset_second_version/'
second_test = second_final_dataset + 'test/'
second_train = second_final_dataset + 'train/'
second_val = second_final_dataset + 'val/'
#sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

### 1. Create train/test split

In [4]:
stopfiles = ['.gitkeep', '.ipynb_checkpoints']
TEST_RATIO = 0.3
VALIDATION_RATIO = 0.3

pictures=list()
for file in os.listdir(cutted_asf_data_directory):
    if not file in stopfiles:
        pictures.append(file)

display(pictures)
total_number_of_pictures = len(pictures)
print(f"total number_of_folder_pictures: {total_number_of_pictures}")
print()
nr_test_pictures = round(TEST_RATIO*total_number_of_pictures)
print(f"number_of_test_folder_pictures: {nr_test_pictures}")
nr_train_pictures = total_number_of_pictures - nr_test_pictures
print(f"number_of_train_folder_pictures: {nr_train_pictures}")
print()
nr_val_pictures = round(VALIDATION_RATIO*nr_train_pictures)
print(f"nr_val_folder_pictures: {nr_val_pictures}")

['S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC',
 'S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC',
 'S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC']

total number_of_folder_pictures: 3

number_of_test_folder_pictures: 1
number_of_train_folder_pictures: 2

nr_val_folder_pictures: 1


In [5]:
test_folders = pictures[0:nr_test_pictures]
print(f"test_folders: {test_folders}")
print()
train_folders = pictures[nr_test_pictures:]
print(f"train_folders: {train_folders}")
print()
val_folders = pictures[nr_test_pictures:nr_test_pictures+nr_val_pictures]
print(f"val_folders: {val_folders}")

test_folders: ['S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC']

train_folders: ['S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC', 'S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC']

val_folders: ['S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC']


### 2. Copy from folders only pictures which has corresponding txt file

In [6]:
def copy_pictures_and_labels(source_path, source_images_folders, destination_path_images, destination_path_labels):
    source_path = source_path
    source_images_folders = source_images_folders
    destination_path_images = destination_path_images
    destination_path_labels = destination_path_labels
    print(destination_path_labels)

    for folder in source_images_folders:
        print()
        print(folder)
        # choose only files with txt annotations
        print(os.listdir(source_path + folder))
        print()
        labels=list()
        images=list()
        for file in os.listdir(source_path + folder):
            if file.endswith(".txt"):
                labels.append(file)
                image_file = file.replace(".txt", ".tif")
                images.append(image_file)
        # copy files - rename them while copying
        # labels
        for file in labels:
            src = source_path + '/' + folder + '/' + file
            dst = destination_path_labels + folder + '_' + file
            shutil.copy(src, dst)
        # pictures
        for file in images:
            src = source_path + '/' + folder + '/' + file
            dst = destination_path_images + folder + '_' + file
            shutil.copy(src, dst)

2.1 Test pictures

In [79]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = final_dataset_images + 'test/',
                        destination_path_labels = final_dataset_labels + 'test/')

../data/7_final_dataset/labels/test/

S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt']



2.2 Train pictures

In [80]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = final_dataset_images + 'train/',
                        destination_path_labels = final_dataset_labels + 'train/')

../data/7_final_dataset/labels/train/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset13.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset7.txt', 'subset8.tif', 'subset8.txt', 'subset9.tif', 'subset9.txt']


S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC
['subset1.tif', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset2.txt', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt', '_subset9.tif']



2.3 Validation pictures

In [81]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = final_dataset_images + 'val/',
                        destination_path_labels = final_dataset_labels + 'val/')

../data/7_final_dataset/labels/val/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset13.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset7.txt', 'subset8.tif', 'subset8.txt', 'subset9.tif', 'subset9.txt']



# 3. Second version of dataset creation

2.1 Test pictures

In [14]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = second_test + 'images/',
                        destination_path_labels = second_test + 'labels/')

../data/8_final_dataset_second_version/test/labels/

S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt']



2.2 Train pictures

In [12]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = second_train + 'images/',
                        destination_path_labels = second_train + 'labels/')

../data/8_final_dataset_second_version/train/labels/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset13.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset7.txt', 'subset8.tif', 'subset8.txt', 'subset9.tif', 'subset9.txt']


S1B_IW_GRDH_1SDV_20210306T015735_20210306T015804_025888_031671_7F65_TC
['subset1.tif', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset2.txt', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt', '_subset9.tif']



2.3 Validation pictures

In [13]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = second_val + 'images/',
                        destination_path_labels = second_val + 'labels/')

../data/8_final_dataset_second_version/val/labels/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset13.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset7.txt', 'subset8.tif', 'subset8.txt', 'subset9.tif', 'subset9.txt']



# OLD

In [65]:
source_path = cutted_asf_data_directory
source_images_folders = test_folders
destination_path_images = final_dataset_images + 'test/'
destination_path_labels = final_dataset_labels + 'test/'
print(destination_path_labels)

for folder in source_images_folders:
    print()
    print(folder)
    # choose only files with txt annotations
    print(os.listdir(source_path + folder))
    print()
    labels=list()
    images=list()
    for file in os.listdir(source_path + folder):
        if file.endswith(".txt"):
            labels.append(file)
            image_file = file.replace(".txt", ".tif")
            images.append(image_file)
    # copy files - rename them while copying
    # labels
    for file in labels:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_labels + folder + file
        shutil.copy(src, dst)
    # pictures
    for file in images:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_images + folder + file
        shutil.copy(src, dst)
    

print(labels)
print()
print(images)

../data/7_final_dataset/labels/test/
S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt']

['subset1.txt', 'subset10.txt', 'subset3.txt', 'subset4.txt', 'subset5.txt', 'subset6.txt', 'subset9.txt']

['subset1.tif', 'subset10.tif', 'subset3.tif', 'subset4.tif', 'subset5.tif', 'subset6.tif', 'subset9.tif']
